In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
def get_num_trials(base_dir):
    dir = base_dir + '/oracle.json'
    with open(dir) as f:
       data = json.load(f)
    return int(data['end_order'][-1])

In [3]:
def create_trials_np(num_trials, base_dir):
    trials_np = np.zeros(0)
    for trial in range(0, num_trials):
        dir = base_dir + '/trial_'
        trials_np = np.append(trials_np, create_np_row(dir, trial)).reshape(-1, 21)
    return trials_np

In [4]:
def create_np_row(dir, trial):
    if len(str(trial)) == 1: 
        trial = '00' + str(trial)
    elif len(str(trial)) == 2:
        trial = '0' + str(trial)
    dir = dir + str(trial) + '/trial.json'
    df = pd.read_json(dir)
    return df
#df, df2 = create_trial_df('tuner1/trial_', 0)

In [5]:
def create_df_row(trials_np, i):
    index = i
    
    hyperparameters = trials_np[i][8]
    
    num_layers = hyperparameters['num_layers']
    d_model = hyperparameters['d_model']
    dff = hyperparameters['dff']
    num_heads = hyperparameters["num_heads"]
    dropout_rate = hyperparameters['dropout_rate']
    warmup_steps = hyperparameters['warmup_steps']
    beta_1 = hyperparameters['beta_1']
    beta_2 = hyperparameters['beta_2']
    epsilon = hyperparameters['epsilon']
    
    metrics = trials_np[i][16]
    
    loss = metrics['loss']['observations'][0]['value'][0]
    accuracy = metrics['accuracy']['observations'][0]['value'][0]
    val_loss = metrics['val_loss']['observations'][0]['value'][0]
    val_accuracy = metrics['val_accuracy']['observations'][0]['value'][0]
    #score = trials_np[i][17]
    best_step = metrics['loss']['observations'][0]['step']
    
    return pd.DataFrame({
        #'trial': index,
        'num_layers': num_layers,
        'd_model': d_model,
        'dff': dff,
        "num_heads": num_heads,
        'dropout_rate': dropout_rate,
        'warmup_steps': warmup_steps,
        'beta_1': beta_1,
        'beta_2': beta_2,
        'epsilon': epsilon,
        'loss': loss, 
        'accuracy' : accuracy,
        'val_loss' : val_loss,
        'val_accuracy' : val_accuracy, 
        #'score': score, 
        'best_step': best_step},
        index = [i])
        
#create_df_row(create_trials_np(num_trials=5, base_dir='tuner2'), 0)

In [6]:
def create_tuner_df(num_trials, trials_np):
    df = create_df_row(trials_np, 0)
    for trial in range(1, num_trials):
        df = pd.concat((df, create_df_row(trials_np, trial)), axis=0)
    return df

In [7]:
def best_trials(tuner_df, num_trials=5):
    df = tuner_df.sort_values(by=['val_accuracy']).tail(num_trials)
    return df

In [8]:
trials_np = np.zeros(0)
base_dir = 'tuner2'
num_trials = get_num_trials(base_dir)
trials_np = create_trials_np(num_trials, base_dir)


tuner_df = create_tuner_df(num_trials, trials_np)
df = best_trials(tuner_df, num_trials=10)

In [9]:
df.head(10)

,num_layers,d_model,dff,num_heads,dropout_rate,warmup_steps,beta_1,beta_2,epsilon,loss,accuracy,val_loss,val_accuracy,best_step
75,4,104,896,12,0.16,3250,0.79,0.975,8.322312e-08,1.175059,0.495020,1.285580,0.505763,24
78,4,120,608,10,0.16,3500,0.79,0.965,7.170623e-08,1.467814,0.387607,1.242491,0.506224,2
39,3,56,1024,13,0.12,4500,0.83,0.960,5.346140e-08,1.339154,0.428968,1.216300,0.506685,6
56,4,80,544,13,0.20,4250,0.87,0.955,9.756409e-08,1.120804,0.526840,1.241187,0.506685,29
31,2,56,896,13,0.22,2500,0.79,0.975,5.443117e-08,1.083400,0.536328,1.306243,0.507146,28
94,2,32,1024,9,0.22,3750,0.81,0.980,3.370615e-08,1.180300,0.497903,1.223569,0.508068,29
98,2,88,736,10,0.04,3000,0.81,0.990,4.219452e-08,1.346760,0.426924,1.219418,0.508068,3
29,2,88,896,14,0.20,2500,0.79,0.975,6.224755e-08,1.358017,0.420843,1.227487,0.508529,5
45,3,128,736,8,0.20,3250,0.79,0.980,6.358079e-08,1.091906,0.534598,1.282422,0.508990,26
64,4,40,864,10,0.22,2500,0.83,0.975,2.072918e-08,1.174359,0.493028,1.237839,0.508990,24
